# Simulation of Local Protocols

This notebook is meant to be used to simulate a local protocol supplemented by one bit of communication. The protocol is simulated using Monte-Carlo simulation.

In [1]:
# Import the required packages
import numpy as np
import math

First, let us define some functions

In [8]:
def random_vector(n):
    """Uniformly distributed random vector"""
    components = [np.random.normal() for i in range(n)]
    r = math.sqrt(sum(x*x for x in components))
    v = [x/r for x in components]
    return v


def dot(vec_1, vec_2):
    """Dot product for two vectors"""
    return vec_1[0] * vec_2[0] + vec_1[1] * vec_2[1] + vec_1[2] * vec_2[2]


def add(vec_1, vec_2):
    """Adding of two vectors"""
    return [vec_1[0] + vec_2[0], vec_1[1] + vec_2[1], vec_1[2] + vec_2[2]]


def subtract(vec_1, vec_2):
    """Subtracting a vector from another"""
    return [vec_1[0] - vec_2[0], vec_1[1] - vec_2[1], vec_1[2] + vec_2[2]]


def alice_protocol(theta, alice, lambda_1):
    """Here is the Toner-Bacon protocol for Alice"""
    return np.sign(dot(alice, lambda_1))


def comm_protocol(theta, alice, lambda_1, lambda_2):
    """Here is the bit of Communication sent by Alice in the Toner-Bacon protocol"""
    return np.sign(dot(alice, lambda_1)) * np.sign(dot(alice, lambda_2))


def bob_protocol(theta, bob, comm, lambda_1, lambda_2):
    """Here is the Toner-Bacon protocol for Bob, given the bit of communication"""
    return np.sign(dot(bob, add(lambda_1, np.multiply(comm, lambda_2))))

def alice_expectation(n, theta, alice):
    """The expectation of the marginals of Alice's output"""
    sum = 0
    for i in range(n):
        lambda_1 = random_vector(3)
        sum += alice_protocol(theta, alice, lambda_1)
    return sum / n

def bob_expectation(n, theta, alice, bob):
    """The expectation of the marginals of Bob's output"""
    sum = 0
    for i in range(n):
        lambda_1 = random_vector(3)
        lambda_2 = random_vector(3)
        comm = comm_protocol(theta, alice, lambda_1, lambda_2)
        sum += bob_protocol(theta, bob, comm, lambda_1, lambda_2)
    return sum / n

def joint_expectation(n, theta, alice, bob):
    """The joint expectation of the marginals of Alice's and Bob's outputs"""
    sum = 0
    for i in range(n):
        lambda_1 = random_vector(3)
        lambda_2 = random_vector(3)
        comm = comm_protocol(theta, alice, lambda_1, lambda_2)
        alice_bit = alice_protocol(theta, alice, lambda_1)
        bob_bit = bob_protocol(theta, bob, comm, lambda_1, lambda_2)
        sum += alice_bit * bob_bit
    return sum / n

Let us try to test the protocol. We print the joint expectation of the simulated protocol and the predicted theoretical value for a singlet.

In [29]:
theta = np.pi/4
n = 100000
alice = random_vector(3)
bob = random_vector(3)

print(joint_expectation(n, theta, alice, bob))
print(dot(alice, bob))

-0.56726
-0.7777107332500339


Here let us now define new protocols. These protocols have the same marginals as the non-maximally entangled state but different joint expectation.

In [27]:
def alice_protocol(theta, alice, lambda_1):
    return np.sign(dot(alice, lambda_1) + np.cos(2 * theta) * alice[2])

def comm_protocol(theta, alice, lambda_1, lambda_2):
    return np.sign(dot(alice, lambda_1)) * np.sign(dot(alice, lambda_2))

def bob_protocol(theta, bob, comm, lambda_1, lambda_2):
    return np.sign(dot(bob, lambda_1 + np.multiply(comm, lambda_2)) + np.cos(2 * theta) * bob[2] - dot(lambda_1, bob))

In [28]:
theta = np.pi/16
alice = random_vector(3)
bob = random_vector(3)
print(alice)
print(bob)

print(alice_expectation(n, theta, alice))
print(np.cos(2 * theta) * alice[2])
print(bob_expectation(n, theta, alice, bob))
print(np.cos(2 * theta) * bob[2])

[0.573890030009261, 0.5413369247807693, -0.6144953761623183]
[-0.25187207746132456, -0.8084313501038054, -0.5319766994565222]
-0.56618
-0.5677197008591899
-0.49092
-0.49148238440078895
